In [1]:
import mysql.connector
import json, os, re
from datetime import datetime
from collections import defaultdict
import pandas as pd
import re
fund_names = ["360 One Mutual Fund", "Aditya Birla Sun Life Mutual Fund", "Axis Mutual Fund", "Angel One Mutual Fund",
              "Bajaj Finserv Mutual Fund", "Bandhan Mutual Fund", "Bank of India Mutual Fund", "BOI Mutual Fund",
              "Baroda BNP Paribas Mutual Fund", "Canara Robeco Mutual Fund", "DSP Mutual Fund", 
              "Edelweiss Mutual Fund", "Franklin Templeton Mutual Fund", "Groww Mutual Fund", 
              "HDFC Mutual Fund", "Helios Mutual Fund", "HSBC Mutual Fund", "ICICI Prudential Mutual Fund",
              "Invesco Mutual Fund", "ITI Mutual Fund", "JM Financial Mutual Fund", "Kotak Mutual Fund",
              "PGIM India Mutual Fund", "LIC Mutual Fund", "Manulife Mutual Fund", "Mirae Asset Mutual Fund", 
              "Motilal Oswal Mutual Fund", "Navi Mutual Fund", "Nippon Mutual Fund", "NJ Mutual Fund", 
              "Old Bridge Mutual Fund", "PPFAS Mutual Fund", "Quantum Mutual Fund", "Quant Mutual Fund", 
              "Samco Mutual Fund", "SBI Mutual Fund", "Shriram Mutual fund", "Sundaram Mutual Fund", "Tata Mutual Fund", 
              "Taurus Mutual Fund", "Trust Mutual Fund", "Union Mutual Fund", "UTI Mutual Fund", "WhiteOak Mutual Fund", "Zerodha Mutual Fund"]

# column_mapping = {
#     "id": "id",
#     "entered_time": "entered_date",
#     "amc_for_month": "amc_for_month",
#     "data_from": "data_from",
#     "amc_name": "amc_name",
#     "main_scheme_name": "main_scheme_name",
#     "fund_name": "fund_name",
#     "benchmark_index": "benchmark_index",
#     "monthly_aaum_date": "monthly_aaum_date",
#     "monthly_aaum_value": "monthly_aaum_value",
#     "scheme_launch_date": "scheme_launch_date",
#     "min_addl_amt": "min_addl_amt",
#     "min_addl_amt_multiple": "min_addl_amt_multiple",
#     "min_amt": "min_amt",
#     "min_amt_multiple": "min_amt_multiple",
#     "entry_load": "entry_load",
#     "exit_load": "exit_load",
#     "alpha": "alpha",
#     "arithmetic_mean_ratio": "arithmetic_mean_ratio",
#     "average_div_yld": "average_div_yield",
#     "average_pb": "average_pb",
#     "average_pe": "average_pe",
#     "avg_maturity": "avg_maturity",
#     "beta": "beta",
#     "correlation_ratio": "correlation_ratio",
#     "downside_deviation": "downside_deviation",
#     "information_ratio": "information_ratio",
#     "macaulay": "macaulay",
#     "mod_duration": "mod_duration",
#     "port_turnover_ratio": "port_turnover_ratio",
#     "r_squared_ratio": "r_squared_ratio",
#     "roe_ratio": "roe_ratio",
#     "sharpe": "sharpe",
#     "sortino_ratio": "sortino_ratio",
#     "std_dev": "std_dev",
#     "tracking_error": "tracking_error",
#     "treynor_ratio": "treynor_ratio",
#     "upside_deviation": "upside_deviation",
#     "ytm": "ytm"
# }

In [ ]:
#mydata
# conn = mysql.connector.connect(
#     host="172.22.225.155",
#     user="cog_mf",
#     password="bnYwFChjLAV2Z%9E",
#     database="cog_mf",
#     port=3306,
#     charset='utf8mb4'
# )

# conn = mysql.connector.connect(
#     host="localhost",   
#     user="root",            
#     password="1234", 
#     database="cog_updated_db",  
#     port=3306,               
#     charset="utf8mb4"
# )

conn = mysql.connector.connect(
    host="localhost",   
    user="root",            
    password="1234", 
    database="data_db",  
    port=3306,               
    charset="utf8mb4"
)

timestamp = datetime.now().strftime("%Y%m%d_%H%M")
output_path = f"fink_data_{timestamp}.xlsx"
data_from_value = ""
month = "JAN25"

with pd.ExcelWriter(output_path, engine="xlsxwriter") as writer:
    queries_dict = {}
    workbook = writer.book
    header_format = workbook.add_format({
        'bold': True,
        'text_wrap': True,
        'valign': 'top',
        'fg_color': '#D7E4BC',
        'border': 1
    })
    
    for funds in fund_names:
        print(funds)
        search_fund = re.sub(r'\bmutual\s*fund\b', '', funds.lower(),flags=re.IGNORECASE).strip()
        print(search_fund)
        query = """
        SELECT *
        FROM mf_common_fund_data
        WHERE amc_name LIKE %s
        AND amc_for_month REGEXP 'MAR|JAN'
        AND data_from = 'fink'
        AND entered_time LIKE "%2025-05-14%";
        """
        df = pd.read_sql(query, conn, params=(f"%{search_fund}%",))

        if df.empty:
            print(f"Skipped (no data): {funds}")
            continue
        
        df.drop_duplicates(inplace=True)
        # df.drop(columns=["id","amc_id","fund_id","entered_time","amc_name","fund_name"], axis=1, inplace=True)
        
        # df.insert(0, 'data_from', data_from_value)
        cols = df.columns.tolist()
        
        sheet_name = "_".join(search_fund.lower().split())[:31]
        df.to_excel(writer, sheet_name=sheet_name, index=False)
        worksheet = writer.sheets[sheet_name]

        for col_num, header in enumerate(df.columns.values):
            worksheet.write(0, col_num, header, header_format)
            worksheet.set_column(col_num, col_num, min(len(header) + 8, 50))
        
print(f"Done! Exported to: {output_path}")
conn.close()

In [ ]:
# #finklivedb
# # query = """
# #             SELECT *
# #             FROM mf_json_common_details
# #             WHERE (monthly_aaum_date REGEXP '2025|FEBRU' OR monthly_aaum_date IS NULL)
# #               AND fund_name LIKE %s
# #         """

# # conn = mysql.connector.connect(
# #     host="172.22.225.155",
# #     user="cog_mf",
# #     password="bnYwFChjLAV2Z%9E",
# #     database="cog_mf",
# #     port=3306,
# #     charset='utf8mb4'
# # )

# conn = mysql.connector.connect(
#     host="localhost",   
#     user="root",            
#     password="1234", 
#     database="cog_updated_db",  
#     port=3306,               
#     charset="utf8mb4"
# )


# timestamp = datetime.now().strftime("%Y%m%d_%H%M")
# output_path = f"feb_live_{timestamp}.xlsx"
# data_from_value = "cog_mf"
# aaum_date = '202501|January 2025'

# with pd.ExcelWriter(output_path, engine="xlsxwriter") as writer:
#     queries_dict = {}
#     workbook = writer.book
#     header_format = workbook.add_format({
#         'bold': True,
#         'text_wrap': True,
#         'valign': 'top',
#         'fg_color': '#D7E4BC',
#         'border': 1
#     })
    
#     for fund in fund_names:
#         search_fund = re.sub(r'\bmutual\s*fund\b', '', fund, flags=re.IGNORECASE).strip()
#         print(search_fund)
#         query = """
#             SELECT *
#             FROM mf_common_fund_data d
#             WHERE d.fund_name LIKE %s
#             AND amc_for_month REGEXP 'FEB'
#             AND data_from REGEXP 'fink'
#         """
#         df = pd.read_sql(query, conn, params=(f"%{search_fund}%",))

#         if df.empty:
#             print(f"Skipped (no data): {fund}")
#             continue
        
#         df = df.sort_values(by='monthly_aaum_date', key=lambda col: col.isnull())
        
    
#         cols = df.columns.tolist()
        
#         sheet_name = "_".join(search_fund.lower().split())[:31]
#         df.to_excel(writer, sheet_name=sheet_name, index=False)
#         worksheet = writer.sheets[sheet_name]

#         for col_num, header in enumerate(df.columns.values):
#             worksheet.write(0, col_num, header, header_format)
#             worksheet.set_column(col_num, col_num, min(len(header) + 8, 50))

            
# print(f"Done! Exported to: {output_path}")
# conn.close()

In [ ]:
# live_data = pd.read_excel("mar_live_20250514_1604.xlsx", sheet_name=None)
# self_data = pd.read_excel("mar_self_20250514_1540.xlsx", sheet_name=None)

# timestamp = datetime.now().strftime("%Y%m%d_%H%M")
# output_path = f"mar_combined{timestamp}.xlsx"

# with pd.ExcelWriter(output_path, engine="xlsxwriter") as writer:
#     for sheet_name in live_data:
#         df_live = live_data[sheet_name].copy()
#         df_live["data_from"] = "livedb"
        
#         df_live.drop_duplicates(inplace=True)
        
#         df_self = self_data.get(sheet_name, pd.DataFrame()).copy()
#         if not df_self.empty:
#             df_self["data_from"] = "mydata"
            
#             df_self.drop_duplicates(inplace=True)
            
#             df_combined = pd.concat([df_self, df_live], axis=1, keys=["self", "live"])
#             df_combined.columns = [f"{i}_{j}" for i, j in df_combined.columns] 
#             # Align columns
#             # df_self = df_self.reindex(columns=df_live.columns, fill_value="")
#             # df_combined = pd.concat([df_live, df_self], ignore_index=True)
#         else:
#             df_combined = df_live

#         df_combined.to_excel(writer, sheet_name=sheet_name[:31], index=False)

# print(f"Saved At: {output_path}")


Saved At: mar_combined20250509_2230.xlsx


In [ ]:
# live_data = pd.read_excel("mar_live_20250509_1821.xlsx", sheet_name=None)
# self_data = pd.read_excel("mar_self_20250509_1820.xlsx", sheet_name=None)

# keys_to_drop = [
#     "id",
#     "document_detail_id",
#     "MainScheme_ID",
#     "scheme_objective",
#     "suitable_for_investors",
#     "benchmark_index_id",
#     "information_ratio",
#     "sortino_ratio",
#     "arithmetic_mean_ratio",
#     "correlation_ratio",
#     "roe_ratio",
#     "downside_deviation",
#     "upside_deviation",
#     "price_per_unit",
#     "creation_unit_size",
#     "offer_price",
#     "min_target_amt",
#     "entered_user",
#     "entered_date",
#     "modified_user",
#     "modified_date",
#     "is_migrated",
#     "is_valid",
#     "close_date",
#     "open_date",
#     "face_value",
#     "type_of_scheme",
#     "Riskometer",
#     "Riskometer_benchmark",
#     "SchemeCategory_level2_ID",
#     "scheme_code",
#     "sai_weblink",
#     "nfo_min_sub_amt"
# ]


# timestamp = datetime.now().strftime("%Y%m%d_%H%M")
# output_path = f"mar_combine{timestamp}.xlsx"

# with pd.ExcelWriter(output_path, engine="xlsxwriter") as writer:
#     for sheet_name in live_data:
#         df_live = live_data[sheet_name].copy()
#         df_live["data_from"] = "livedb"
#         df_live.drop_duplicates(inplace=True)
#         df_live = df_live.sort_values(by="main_scheme_name")
#         df_self.drop(['amc_for_month', 'entered_time', 'entry_load','exit_load'], axis=1, inplace=True)
        

#         df_self = self_data.get(sheet_name, pd.DataFrame()).copy()
#         if not df_self.empty:
#             df_self["data_from"] = "mydata"
#             df_self.drop_duplicates(inplace=True)
#             df_self.drop(['amc_for_month', 'entered_time', 'entry_load','exit_load'], axis=1, inplace=True)
#             df_self = df_self.sort_values(by="main_scheme_name")

#             df_combined = pd.concat([df_self, df_live], axis=1, keys=["self", "live"])

#             # interleaved_cols = []
#             # for col in df_self.columns:
#             #     interleaved_cols.append(("self", col))
#             #     interleaved_cols.append(("live", col))

#             # df_combined = df_combined[interleaved_cols]
#             df_combined.columns = [f"{i}_{j}" for i, j in df_combined.columns]

#         else:
#             df_combined = df_live

#         df_combined.to_excel(writer, sheet_name=sheet_name[:31], index=False)

# print(f"Saved at: {output_path}")


In [ ]:
# import pandas as pd
# from datetime import datetime

# def normalize_names(df):
#     df['main_scheme_name'] = df['main_scheme_name'].str.strip().str.lower()
#     return df

# timestamp = datetime.now().strftime("%Y%m%d_%H%M")
# output_path = f"aligned_comparison_{timestamp}.xlsx"

# keys_to_drop = [
#     "id",
#     "document_detail_id",
#     "MainScheme_ID",
#     "scheme_objective",
#     "suitable_for_investors",
#     "benchmark_index_id",
#     "information_ratio",
#     "sortino_ratio",
#     "arithmetic_mean_ratio",
#     "correlation_ratio",
#     "roe_ratio",
#     "downside_deviation",
#     "upside_deviation",
#     "price_per_unit",
#     "creation_unit_size",
#     "offer_price",
#     "min_target_amt",
#     "entered_user",
#     "entered_date",
#     "modified_user",
#     "modified_date",
#     "is_migrated",
#     "is_valid",
#     "close_date",
#     "open_date",
#     "face_value",
#     "type_of_scheme",
#     "Riskometer",
#     "Riskometer_benchmark",
#     "SchemeCategory_level2_ID",
#     "scheme_code",
#     "sai_weblink",
#     "nfo_min_sub_amt"
# ]

# live_data = pd.read_excel("feb_live_20250509_1823.xlsx", sheet_name=None)
# self_data = pd.read_excel("feb_self_20250509_1825.xlsx", sheet_name=None)

# with pd.ExcelWriter(output_path, engine="xlsxwriter") as writer:
#     for sheet_name in live_data:
#         df_live = live_data[sheet_name].copy()
#         df_self = self_data.get(sheet_name, pd.DataFrame()).copy()

#         if not df_self.empty:
#             df_self = normalize_names(df_self)
#             df_live = normalize_names(df_live)

#             df_self['data_from'] = 'mydata'
#             df_live['data_from'] = 'livedb'

#             df_self = df_self.drop_duplicates().sort_values(by="main_scheme_name")
#             df_live = df_live.drop_duplicates().sort_values(by="main_scheme_name")

#             df_self.drop(['amc_for_month', 'entered_time', 'entry_load', 'exit_load'], axis=1, inplace=True, errors='ignore')
#             df_self.drop(keys_to_drop, axis=1, inplace=True, errors='ignore')
#             df_live.drop(keys_to_drop, axis=1, inplace=True, errors='ignore')

#             # Remove duplicate main_scheme_name entries
#             df_self = df_self.drop_duplicates(subset=["main_scheme_name"])
#             df_live = df_live.drop_duplicates(subset=["main_scheme_name"])

#             all_names = pd.Series(
#                 pd.concat([df_self["main_scheme_name"], df_live["main_scheme_name"]]).unique()
#             ).sort_values().reset_index(drop=True)

#             df_self_aligned = df_self.set_index("main_scheme_name").reindex(all_names).reset_index()
#             df_live_aligned = df_live.set_index("main_scheme_name").reindex(all_names).reset_index()

#             # Rename columns (handling main_scheme_name correctly)
#             df_self_aligned.columns = [f"{col}_self" if col != "main_scheme_name" else col for col in df_self_aligned.columns]
#             df_live_aligned.columns = [f"{col}_live" if col != "main_scheme_name" else col for col in df_live_aligned.columns]

#             # Drop 'main_scheme_name' only if it exists in df_live_aligned
#             if 'main_scheme_name' in df_live_aligned.columns:
#                 df_combined = pd.concat([df_self_aligned, df_live_aligned.drop(columns=["main_scheme_name"])], axis=1)
#             else:
#                 df_combined = pd.concat([df_self_aligned, df_live_aligned], axis=1)
#         else:
#             df_live['data_from'] = 'livedb'
#             df_combined = df_live

#         df_combined.to_excel(writer, sheet_name=sheet_name[:31], index=False)

# print(f"Saved at: {output_path}")


Saved at: aligned_comparison_20250521_1136.xlsx


In [16]:
# 

In [ ]:
# 

In [ ]:
# live_data = pd.read_excel("aligned_comparison_20250511_1305.xlsx", sheet_name=None)
# combined_parts = []

# for sheet_name in sorted(live_data):
#     df = live_data[sheet_name]
#     combined_parts.append(df)
#     blank_rows = pd.DataFrame(columns=df.columns, index=range(5))
#     combined_parts.append(blank_rows)

# final_df = pd.concat(combined_parts, ignore_index=True)
# final_df.to_excel("jan_all_dataset.xlsx", index=False)

In [ ]:
# cols = [
#     "main_scheme_name_self", "amc_name_self", "fund_name_self", "benchmark_index_self", "min_addl_amt_self", 
#     "min_addl_amt_multiple_self", "min_amt_self", "min_amt_multiple_self", "monthly_aaum_value_self", 
#     "scheme_launch_date_self", "ytm_self", "port_turnover_ratio_self", "mod_duration_self", "avg_maturity_self", 
#     "macaulay_self", "sharpe_self", "std_dev_self", "beta_self", "tracking_error_self", "r_squared_ratio_self", 
#     "average_div_yield_self", "average_pb_self", "average_pe_self", "alpha_self", "treynor_ratio_self", 
#     "main_scheme_name_live", "amc_name_live", "fund_name_live", "benchmark_index_live", "min_addl_amt_live", 
#     "min_addl_amt_multiple_live", "min_amt_live", "min_amt_multiple_live", "monthly_aaum_value_live", 
#     "scheme_launch_date_live", "ytm_live", "port_turnover_ratio_live", "mod_duration_live", "avg_maturity_live", 
#     "macaulay_live", "sharpe_live", "std_dev_live", "beta_live", "tracking_error_live", "r_squared_ratio_live", 
#     "average_div_yield_live", "average_pb_live", "average_pe_live", "alpha_live", "treynor_ratio_live", 
#     "main_scheme_name_match", "amc_name_match", "fund_name_match", "benchmark_index_match", "min_addl_amt_match", 
#     "min_addl_amt_multiple_match", "min_amt_match", "min_amt_multiple_match", "monthly_aaum_value_match", 
#     "scheme_launch_date_match", "ytm_match", "port_turnover_ratio_match", "mod_duration_match", "avg_maturity_match", 
#     "macaulay_match", "sharpe_match", "std_dev_match", "beta_match", "tracking_error_match", "r_squared_ratio_match", 
#     "average_div_yield_match", "average_pb_match", "average_pe_match", "alpha_match", "treynor_ratio_match"
# ]


# df = pd.read_excel("march_data_compare.xlsx")

# groups = defaultdict(dict)

# for col in df.columns:
#     if col.endswith("_match"):
#         base = col.replace("_match", "")
#         groups[base]["match"] = col
#     elif col.endswith("_self"):
#         base = col.replace("_self", "")
#         groups[base]["self"] = col
#     elif col.endswith("_live"):
#         base = col.replace("_live", "")
#         groups[base]["live"] = col


# ordered_cols = []
# for base in groups:
#     for suffix in ["self", "live", "match"]:
#         if suffix in groups[base]:
#             ordered_cols.append(groups[base][suffix])


# df_reordered = df[ordered_cols]

# # print(df_reordered.columns.tolist())

In [ ]:
# df_reordered.to_excel("interlave_march.xlsx", index=False)

In [ ]:
# 

In [ ]:
# 

In [ ]:
# live_data = pd.read_excel(os.path.join(os.getcwd(),"mar_live_20250514_1604.xlsx"), sheet_name=None)
# self_data = pd.read_excel(os.path.join(os.getcwd(),"mar_self_20250514_1618.xlsx"), sheet_name=None)


# fund_names = list(live_data.keys())
# timestamp = datetime.now().strftime("%Y%m%d_%H%M")
# output_path = f"march_self_live_{timestamp}.xlsx"

# with pd.ExcelWriter(output_path, engine="xlsxwriter") as writer:
#     workbook = writer.book
#     header_format = workbook.add_format({
#         'bold': True,
#         'text_wrap': True,
#         'valign': 'top',
#         'fg_color': '#D7E4BC',
#         'border': 1
#     })

#     for fund in fund_names:
#         df_live = live_data[fund].copy()

#         df_live["data_from"] = "livedb"

#         df_offline = self_data.get(fund, pd.DataFrame()).copy()
#         if df_offline.empty:
#             df_combined = df_live
#         else:
#             # df_offline["data_from"] = "mydata"
#             df_offline_renamed = df_offline.rename(columns=column_mapping)
#             df_offline_aligned = df_offline_renamed.reindex(columns=df_live.columns)
#             # Combine live + offline
#             df_combined = pd.concat([df_live, df_offline_aligned], ignore_index=True)
#         df_combined.to_excel(writer, sheet_name=fund, index=False)
#         worksheet = writer.sheets[fund]

#         for col_num, header in enumerate(df_combined.columns):
#             worksheet.write(0, col_num, header, header_format)
#             worksheet.set_column(col_num, col_num, min(len(header) + 8, 50))

# print("Done!")


Done!


In [ ]:
# """
# SELECT 
#     d.*,
#     COALESCE(d.benchmark_index, bi.benchmark_index) AS final_benchmark_index,
#     bi.riskometer_benchmark,
#     MAX(CASE WHEN l.type = 'Entry' THEN l.comment END) AS entry_load,
#     MAX(CASE WHEN l.type = 'Exit' THEN l.comment END) AS exit_load
# FROM mf_json_common_details d
# LEFT JOIN mf_json_loads l
#   ON d.MainScheme_ID = l.MainScheme_ID
#  AND d.document_detail_id = l.document_detail_id
# LEFT JOIN mf_json_benchmark_indices bi
#   ON d.MainScheme_ID = bi.MainScheme_ID
#  AND d.document_detail_id = bi.document_detail_id
# WHERE d.fund_name LIKE %s
#   AND (d.monthly_aaum_date REGEXP '202502|FEBRUARY' OR d.monthly_aaum_date IS NULL)
# GROUP BY d.id;
# """

In [ ]:
import os, json, pprint

def get_json_paths(root_folder):
    json_paths = []
    for folder_name in os.listdir(root_folder):
        folder_path = os.path.join(root_folder, folder_name)
        if os.path.isfile(folder_path) and folder_path.endswith(".json"):
            json_paths.append(folder_path)
    return json_paths

def load_all_records(json_paths):
    all_records = {}
    for path in json_paths:
        try:
            with open(path, "r", encoding="utf-8") as f:
                data = json.load(f)
                records = data.get("records", [])
                name = " ".join(records[0].get("value", {}).get("amc_name", "").split(" ")[:2])
                if name not in all_records:
                    all_records[name] = records
                else:
                    name += " 2"
                    all_records[name] = records
        except Exception as e:
            print(f"Error reading {path}: {e}")
    return all_records

def null_if_empty(value):
    if value in ("", None):
        return None
    return value

In [5]:
base_dir = os.path.join(os.path.dirname(os.getcwd()), "sql_learn", "json","feb25")
json_paths = get_json_paths(base_dir)
combined_records = load_all_records(json_paths)

In [ ]:
pprint.pprint(list(combined_records.keys()))

In [13]:
combined_records["360 ONE"]

[{'value': {'amc_name': '360 ONE Asset Management Limited',
   'benchmark_index': ['BSE 500 TRI'],
   'field_location': [{'amc_name': '6',
     'benchmark_index': '6',
     'count': 7,
     'fund_manager_name': '6',
     'fund_manager_total_exp': '6',
     'load_entry': '6',
     'load_exit': '6',
     'main_scheme_name': '6',
     'metrics_beta': '6',
     'metrics_port_turnover_ratio': '6',
     'metrics_r_squared_ratio': '6',
     'metrics_sharpe': '6',
     'metrics_std_dev': '6',
     'metrics_treynor_ratio': '6',
     'min_addl_amt': '6',
     'min_addl_amt_multiple': '6',
     'min_amt': '6',
     'min_amt_multiple': '6',
     'monthly_aaum_date': '6',
     'monthly_aaum_value': '6',
     'mutual_fund_name': '6',
     'scheme_launch_date': '6'}],
   'fund_manager': [{'managing_fund_since': '',
     'name': 'Mayur Patel',
     'qualification': '',
     'total_exp': '20 Years'},
    {'managing_fund_since': '',
     'name': 'Ashish Ongari',
     'qualification': '',
     'total_exp

In [ ]:
keys = [
    "amc_name", "main_scheme_name", "fund_name","benchmark_index", "monthly_aaum_date", 
    "monthly_aaum_value", "scheme_launch_date","min_addl_amt", "min_addl_amt_multiple", 
    "min_amt", "min_amt_multiple", "entry_load", "exit_load",
    "alpha", "arithmetic_mean_ratio", "average_div_yield", "average_pb", "average_pe", "avg_maturity",
    "beta", "correlation_ratio", "downside_deviation", "information_ratio", "macaulay",
    "mod_duration", "port_turnover_ratio", "r_squared_ratio", "roe_ratio", "sharpe", "sortino_ratio",
    "std_dev", "tracking_error", "treynor_ratio", "upside_deviation", "ytm"
]
details = {}

entry_load = next((l.get("comment") for l in details.get("load", []) if l.get("type") == "entry_load"), "")
exit_load = next((l.get("comment") for l in details.get("load", []) if l.get("type") == "exit_load"), "")

raw_values = [
    details.get("amc_name", ""),
    details.get("main_scheme_name", ""),
    details.get("mutual_fund_name", ""),
    ", ".join(details.get("benchmark_index", [])) if isinstance(details.get("benchmark_index"), list) else details.get("benchmark_index", ""),
    details.get("monthly_aaum_date", ""),
    details.get("monthly_aaum_value", ""),
    details.get("scheme_launch_date", ""),
    details.get("min_addl_amt", ""),
    details.get("min_addl_amt_multiple", ""),
    details.get("min_amt", ""),
    details.get("min_amt_multiple", ""),
    entry_load,
    exit_load
] + [
    next((m.get("value") for m in details.get("metrics", []) if m.get("name") == key), "")
    for key in keys[17:]
]

values = [null_if_empty(v) for v in raw_values]
